In [62]:
from lxml import etree
book=etree.parse('src/book.6.xml',etree.XMLParser(remove_blank_text=True)).getroot()
# re-indentation: no need to do anything
if 0:
    # verse at the beginning of the paragraph, first line as running text
    beginVerse=['IV.66','XVII.161','XVII.210','XVII.212','XVII.218','XVII.230','XVII.237','V.27',]
    for a in beginVerse:
        pp=book.findall('.//p[@anchor="'+a+'"]')
        assert len(pp)==1
        p=pp[0]
        print(len(p),p[0].tag)
        assert (len(p)==1 and p[0].tag=='span') or (len(p)==2 and p[0].tag=='printed_page' and p[1].tag=='span')
        v=p.getnext()
        assert v.tag=='verse'
        for _ in reversed(p): v.insert(0,_)
        v.attrib['y']=p.attrib['y']
        
if 0:
    # last line of paragraph broken off
    endParaFragment=['II.38','II.42','II.55','II.70','II.74','III.73','III.122','III.127',
                     'IV.87',
                     'XVII.232','XVII.238','XVII.248','XVII.260','XVII.272','XVII.281',
                     'XVII.291','XVIII.7','XX.29','XX.42']
    for anchor in endParaFragment:
        pp=book.findall('.//p[@anchor="'+anchor+'"]')
        assert len(pp)==1
        para=pp[0]
        # print(anchor,para,para.sourceline)
        paraAfter=para.getnext()
        while not 'anchor' in paraAfter.attrib: paraAfter=paraAfter.getnext()
        para1=paraAfter.getprevious()
        para0=para1.getprevious()
        #print(paraAfter.attrib['anchor'])
        #print(paraLast[-1].tag)
        print(anchor,len(para1),[e.tag for e in para1])
        print('  ',para0[-1].text)
        print('  ',para1[0].text)
        assert(para0[-1].tag==para1[0].tag)
        para0[-1].text+=' '+para1[0].text
        para1.remove(para1[0])
        for e in para1: para0.append(e)
        # print('   ',len(para0),[e.tag for e in para0])
        # print(f'{para[-1].tag=}')

if 0:
    for p in book.findall('.//p'):
        if len(p)>1 or len(p)==0: continue
        e0=p[0]
        if e0.tag not in ['em','span']: continue
        assert e0.text is not None
        if len(e0.text)>100: continue
        #print(e0.text)
        if e0.text.startswith('[') and e0.text.endswith(']') and e0.text[-2]!=':': print(e0.text)
        elif 'family' in e0.attrib: print(e0.attrib['family'],e0.text)
        
if 1:
    for p in book.findall('.//p'):
        if 'num' not in p.attrib: continue
        if len(p)>0: continue
        #print(p.attrib['num'])
        # p.text='[foo]'
        p.append(e:=etree.Element('span'))
        e.text='[foo]'
        

open('src/book.6a.xml','w').write(etree.tostring(book,encoding='unicode',pretty_print=True))

3591932

In [22]:
from lxml import etree
ix=etree.parse('src/index.xml',etree.XMLParser(remove_blank_text=True)).getroot()

def splitEntry(entry,splitter):
    assert splitter.tag=='Xspan'
    ixs=entry.index(splitter)
    ent2=etree.Element('entry',page_id=entry.attrib['page_id'],title=splitter.text.strip())
    ent2[:]=entry[ixs+1:]
    entry.remove(splitter)
    entry.addnext(ent2)
    return ent2

for e in ix.findall('.//Xspan'):
    assert (ent:=e.getparent()).tag=='entry'
    if e.text.endswith('('):
        assert (n:=e.getnext()).tag=='em'
        assert (nn:=e.getnext().getnext()).tag=='span'
        e.text=e.text[:-1]
        ent2=splitEntry(ent,e)
        ent2.insert(0,sp:=etree.Element('span'))
        sp.text='('
    else:
        assert (n:=e.getnext()).tag=='ref'
        splitEntry(ent,e)
        
        
        
    # assert e.tag=='entry'
    # print(e)
    

open('src/index.2.xml','w').write(etree.tostring(ix,encoding='unicode',pretty_print=True))

558539